In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
with open('./models/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/lemi/Documents/pystd_venv/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/lemi/Documents/pystd_venv/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
% script 
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
df = read_data('../data/raw/yellow_tripdata_2023-03.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [6]:
mean_y_pred = np.mean(y_pred)

In [7]:
var_y_pred = np.var(y_pred)

In [8]:
std_y_pred = np.sqrt(var_y_pred)
print(std_y_pred)

6.247488852238703


In [11]:
df_result = pd.DataFrame()

In [9]:
year = 2023
month = 3

In [12]:
df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [14]:
df_result['y_pred'] = y_pred

In [15]:
df_result

,ride_id,y_pred
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486
...,...,...
3316211,2023/03_3403761,11.952635
3316212,2023/03_3403762,20.049958
3316213,2023/03_3403763,11.595336
3316214,2023/03_3403764,13.113178


In [17]:
output_file = './prediction.parquet'

In [18]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [19]:
year = 2023
month = 4

In [21]:
url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdataz-{year:04d}-{month:02d}.parquet'